3-1. 네이버 책 검색 API 호출하기

In [2]:
#.env파일 읽어오기(해당 파일 실행 중에 .env를 만들거나 수정 하면 restart 하면 됨)
from dotenv import load_dotenv
import os

load_dotenv() #env파일을 메모리상에 적재 시켜주는 함수

#CLIENT_SECRET,CLIENT_ID 가져와서 변수에 담음
CLIENT_ID = os.getenv("CLIENT_ID")
print(CLIENT_ID[:4])

CLIENT_SECRET = os.getenv("CLIENT_SECRET")
print(CLIENT_SECRET[:4])


6uSt
tldx


In [ ]:
# 캡차 키 발급 요청 (실행시 결과 키 값:scwWeA9fEZRO0S5v)
import urllib.request

code = "0" # code="0"은 '키 발급', code="1"은 '사용자 입력값 비교'
url = "https://openapi.naver.com/v1/captcha/nkey?code=" + code
request = urllib.request.Request(url)
#요청 헤더 설정 (CLIENT_ID,CLIENT_SECRET = env 파일에 작성한 것 읽어오기) 
request.add_header("X-Naver-Client-Id",CLIENT_ID) 
request.add_header("X-Naver-Client-Secret",CLIENT_SECRET)

# API 호출 및 응답 처리
response = urllib.request.urlopen(request)
rescode = response.getcode() # HTTP 상태 코드 확인
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))# 응답 데이터를 읽어서 'utf-8' 디코딩
else:
    print("Error Code:" + rescode)

{"key":"56H9VonQyDivsUps"}


In [ ]:
# 캡차 이미지 요청
import urllib.request

key = "56H9VonQyDivsUps" # 캡차 Key 값
url = "https://openapi.naver.com/v1/captcha/ncaptcha.bin?key=" + key

#API 요청 객체 생성 및 헤더 설정
request = urllib.request.Request(url)
# .env에서 가져온 인증 정보를 헤더에 추가
request.add_header("X-Naver-Client-Id",CLIENT_ID)
request.add_header("X-Naver-Client-Secret",CLIENT_SECRET)

#API 호출 및 응답 확인
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    print("캡차 이미지 저장")
    response_body = response.read()
    # 'wb' = 이미지 같은 이진 파일을 저장할 때 필수
    with open('img/captcha.jpg', 'wb') as f: #img폴더 아래에 captcha이미지 생성 및 저장 
        f.write(response_body)
else:
    print("Error Code:" + rescode)

캡차 이미지 저장


In [ ]:
#requests를 사용하는 코드로 변경
#  사용자 입력값 검증 요청
import requests

code = "1"
key = "56H9VonQyDivsUps"
value = "N256KAF"
req_header={ 
    "X-Naver-Client-Id": CLIENT_ID,
    "X-Naver-Client-Secret" : CLIENT_SECRET
}#req_header라는 딕셔너리 생성

# 요청 파라미터 설정 (Query String)
req_param = {
    "code" : code,
    "key" : key,
    "value" : value #이미지에서 나온 숫자 값
} # <=url = "https://openapi.naver.com/v1/captcha/nkey?code=" + code + "&key=" + key + "&value=" + value 해당 코드를 쿼리스트링으로 변경

url = "https://openapi.naver.com/v1/captcha/nkey"

#오류 날수 있을 경우를 위해 try구문 작성
try:
    res = requests.get(url, headers=req_header, params=req_param) #GET 방식으로 요청 , params 인자가 딕셔너리를 자동으로 URL 뒤에 붙임
    res.raise_for_status() #오류(4xx,5xx)가 나면 예외를 발생시킴(발생 시 except로 이동해서 오류메시지 출력)

    print(res.text)
except requests.exceptions.RequestException as e:
    print(f'Status code : {res.status_code}')
    print(f'Error 발생 : {e}')

{"result":true,"responseTime":123.41}


1. Book 검색하기

In [46]:
import requests
from pprint import pprint #(from pprint(모듈이름-pprint.py) import pprint(pprint라는 모듈 안에 있는 pprint라는 함수이름) )

def search_books(자바):
    #네이버 책 검색 API를 호출하여 결과를 반환하는 함수

    url = 'https://openapi.naver.com/v1/search/book.json'

    headers = {
    'X-Naver-Client-Id': CLIENT_ID,
    'X-Naver-Client-Secret': CLIENT_SECRET,
    }

    # query string 문자열을 dict 선언
    payload = {
        'query': '자바', ## 검색어를 인자로 받아 동적 처리
        'display': 100, #한번에 표시할 검색 결과 개수
        'sort': 'sim' #검색 결과 정렬 방법(정확도 순으로 내림차순 정렬, 지정 안해도 디폴트 값은 sim)
    }


    # API 요청
    res = requests.get(url, params=payload, headers=headers)
    if res.status_code == 200:
        return res.json()['items']
    else:
        print(f"Error: {res.status_code}")
        return []

2. 검색어로 책 목록 가져오고 json 파일로 저장하기

In [48]:
#검색어로 책 목록 가져오기
search_term = '자바'
items_data = search_books(search_term)
print(items_data[:2])


[{'title': '파워자바 (Fully Updated)', 'link': 'https://search.shopping.naver.com/book/catalog/58005600954', 'image': 'https://shopping-phinf.pstatic.net/main_5800560/58005600954.20251205080227.jpg', 'author': '천인국^천명준', 'discount': '32400', 'publisher': '인피니티북스', 'pubdate': '20260105', 'isbn': '9791175330061', 'description': "20년 초장기 자바프로그래밍 베스트셀러! \n수많은 자바개발자를 탄생시킨 바로 그 책이 Fully Updated 되어 새롭게 등장했다!\n강의자에게는 핵심을 정확히 찌르는 강의를, 학습자에게는 단계별로 따라갈 수 있는 최적의 학습 경험 제공!\n\n피할 수 없는 인공지능 시대, 이제 더 이상 어렵게 배우지 마라!\n『파워 자바 4판』 한 권이면 제대로, 쉽게 배울 수 있다!\n\n새로운 프로그래밍 언어가 끊임없이 등장하는 오늘날에도 ‘자바’는 여전히 가장 사랑받는 언어다. 지난 20년간 독보적인 지지와 인기를 얻은 ‘파워 자바’는 개정을 거듭하며 그 명성을 함께 했다. 특히 국내 IT 계열 베스트셀러 작가인 천인국 교수가 집필하며 탄탄한 설명과 풍부한 내용으로 신뢰받아 왔다. 이번 『파워 자바 4판』은 그 20년 내공을 기반으로 완전히 새롭게 업데이트되었다.\n\n『파워 자바 4판』은 자바의 기본 개념부터 객체 지향, GUI, 멀티 스레딩, 함수형 프로그래밍까지 단 한 권으로 배우는 만능 학습서이다. 학습자가 스스로 사고하며 응용력을 기를 수 있도록 다양한 프로젝트로 구성하여 전면 개정되었다. 또한 필요 이상으로 난이도 높은 타 교재와 달리 훨씬 이해하기 쉽고 알찬 구성으로 초보자도 재미있게 배울 수 있도록 차별화하였다.\n\n객체 지향과 같은 핵심 개념은 GUI LAB에서 실습을 통해 직

In [ ]:
#data/books.json 파일 생성
#책 목록 json 파일로 저장하기
import json
import os

#C:\_Python\Python_WebScraping_Analysis\연습문제_웹스크래핑\data --> 연습문제_웹스크래핑 안에 data폴더 새로 생성
os.makedirs('data', exist_ok=True)

# 2. 파일 저장 실행
with open('data/books.json', 'w', encoding='utf-8') as f:
    json.dump(items_data, f, ensure_ascii=False, indent=4)

print("1. JSON 파일 저장 완료: data/books.json")

1. JSON 파일 저장 완료: data/books.json


3. books.json 파일을 Pandas DataFrame로 저장하기

In [40]:
import pandas as pd

# [질문 2] books.json 파일을 Pandas DataFrame으로 저장하기
# 저장했던 json 파일을 다시 읽어오기
with open('data/books.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)

# 데이터 변환
df = pd.DataFrame(items_data)

print(json_data)

[{'title': 'Life_Query', 'link': 'https://search.shopping.naver.com/book/catalog/55065001919', 'image': 'https://shopping-phinf.pstatic.net/main_5506500/55065001919.20250601111412.jpg', 'author': '', 'discount': '70280', 'publisher': 'Xlibris US', 'pubdate': '20250505', 'isbn': '9798369445099', 'description': 'What if the remnants of discarded dreams could be reshaped into something new?  What if every failed attempt, every half-reached enterprise, and every fleeting connection carried the weight of something more- something is waiting to be reborn.'}, {'title': 'Life_Query', 'link': 'https://search.shopping.naver.com/book/catalog/55065002718', 'image': 'https://shopping-phinf.pstatic.net/main_5506500/55065002718.20250601111443.jpg', 'author': '', 'discount': '38830', 'publisher': 'Xlibris US', 'pubdate': '20250505', 'isbn': '9798369445105', 'description': 'What if the remnants of discarded dreams could be reshaped into something new?  What if every failed attempt, every half-reached e

4. 검색어로 찾은 책 목록 출력하기

In [ ]:
# [질문 3] 검색어로 찾은 책 목록 출력하기
print("출력 결과 :")
# 상위 5개 데이터를 표 형태로 3개만 출력 
display(df.head(3))

출력 결과 :


,title,link,image,author,discount,publisher,pubdate,isbn,description
0,파워자바 (Fully Updated),https://search.shopping.naver.com/book/catalog...,https://shopping-phinf.pstatic.net/main_580056...,천인국^천명준,32400,인피니티북스,20260105,9791175330061,20년 초장기 자바프로그래밍 베스트셀러! \n수많은 자바개발자를 탄생시킨 바로 그 ...
1,자바 (Seventh Edition),https://search.shopping.naver.com/book/catalog...,https://shopping-phinf.pstatic.net/main_324670...,"Horstmann, Cay S.",31500,한티미디어,20140303,9788964211830,▶ 이 책은 자바를 다룬 이론서입니다. 자바의 기초적이고 전반적인 내용을 학습할 수...
2,자바 마스터 (5단계 반복 학습으로 배우는),https://search.shopping.naver.com/book/catalog...,https://shopping-phinf.pstatic.net/main_580779...,문석재,30600,한빛아카데미,20251208,9791173400391,자바 프로그래밍의 본질과 핵심 원리를\n단계적으로 마스터하는 자바 기본서!\n\n『...


5. 검색어로 찾은 책 목록 중 가격이 2만원 이상인 책만 출력

In [44]:
# 조건1: title, author, discount, publisher, pubdate 컬럼만 출력
# 조건2: 가격은 descending (내림차순), index 초기화

#  'discount' 컬럼을 숫자형(int)으로 변환 
df['discount'] = pd.to_numeric(df['discount'], errors='coerce')

#  가격이 20,000원 이상인 데이터 필터링
df_high_price = df[df['discount'] >= 20000].copy()

#  필요한 컬럼만 선택 및 정렬
df_high_price = df_high_price[['title', 'author', 'discount', 'publisher', 'pubdate']]
df_high_price = df_high_price.sort_values(by='discount', ascending=False).reset_index(drop=True)

print("4. 가격이 2만원 이상인 책 목록:")
display(df_high_price.head())



4. 가격이 2만원 이상인 책 목록:


,title,author,discount,publisher,pubdate
0,기본기가 탄탄한 자바 개발자 (모던 자바의 새로운 기능을 배우고 자바 전문가가 되는 법),"벤저민 J. 에번스^Clark, Jason^마르테인 페르뷔르흐",40500,제이펍,20240814
1,자바 최적화 (클라우드 시대의 자바 성능 튜닝을 위한 실용적인 기법),"벤저민 J. 에번스^Gough, James",37800,한빛미디어,20250530
2,헤드퍼스트 자바 (자바로 여는 무한한 객체지향의 세계!),캐시 시에라^버트 베이츠^트리샤 지,37800,한빛미디어,20240102
3,초보 개발자를 위한 자바 (한 권으로 배우는 자바 마스터 가이드 북),이병승,34200,영진닷컴,20231212
4,이것이 자바다 (교육 현장에서 가장 많이 쓰이는 JAVA 프로그래밍 기본서),신용권^임경균,34200,한빛미디어,20240401


6. 검색어로 찾은 책 목록 중에서 출판사가 “인피니티북스”인 책만 출력하기

In [43]:
# 조건: image, description 컬럼은 제외한 모든 컬럼 출력
# 조건: index 는 초기화

# 출판사가 '인피니티북스'인 데이터 필터링
df_infinity = df[df['publisher'] == '인피니티북스'].copy()

# 'image'와 'description' 컬럼 제외 (drop 사용)
df_infinity = df_infinity.drop(columns=['image', 'description'])

#  index 초기화
df_infinity = df_infinity.reset_index(drop=True)

print("\n5. 출판사가 '인피니티북스'인 책 목록:")
display(df_infinity)


5. 출판사가 '인피니티북스'인 책 목록:


,title,link,author,discount,publisher,pubdate,isbn
0,파워자바 (Fully Updated),https://search.shopping.naver.com/book/catalog...,천인국^천명준,32400,인피니티북스,20260105,9791175330061
1,자바 프로그래밍,https://search.shopping.naver.com/book/catalog...,임좌상^조용주,26100,인피니티북스,20210217,9791185578774
